In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [2]:
def reset_tf():
    global sess
    sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [3]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [4]:
class HyperParameters():
    # maximum number of symbols in an input sequence
    max_sequence_length = 40

    # number of symbols in vocabulary
    # (symbols are expected to be in range(vocab_size))
    vocab_size = 10000

    # number of dimensions in input embeddings
    embedding_size = 256
    
    # number of dimensions in hidden state
    rnn_hidden_state_size = 256
    
    # feed-forward hidden state size
    ff_hidden_state_size = 512
    
    # number of sequences per batch
    batch_size = 256
    
    # dropout rate
    dropout_rate = 0.1
    
    # number of parsing threads in data pipeline
    dataset_pipeline_parallel_calls = 4
    
    # size of prefetch in data pipeline
    dataset_pipeline_prefetch = batch_size * 16
    
    # shuffle buffer size
    dataset_pipeline_shuffle_buffer_size = 10000

hp = HyperParameters()

In [5]:
def parse_example(example_proto, max_sequence_length=hp.max_sequence_length):
    features = {
        'inputs': tf.VarLenFeature(tf.int64),
        'word_endings': tf.VarLenFeature(tf.int64),
        'targets': tf.VarLenFeature(tf.int64)
    }
    
    parsed = tf.parse_single_example(example_proto, features)
    
    def convert_and_pad(sparse_tensor):
        result = tf.sparse_tensor_to_dense(sparse_tensor)
        # TODO: properly ignore elements which are too large (right now we just clip)
        result = result[:max_sequence_length]
        result = tf.pad(result, [[0, max_sequence_length - tf.shape(result)[0]]])
        return result
    
    return (convert_and_pad(parsed['inputs']),
            tf.shape(parsed['inputs'])[0],
            convert_and_pad(parsed['word_endings']),
            convert_and_pad(parsed['targets']))

In [6]:
reset_tf()

# Data pipeline
# -------------

dataset_filenames = tf.placeholder(tf.string, shape=[None])

dataset = tf.data.TFRecordDataset(dataset_filenames)
dataset = dataset.map(parse_example, 
                      num_parallel_calls = hp.dataset_pipeline_parallel_calls)
dataset = dataset.shuffle(hp.dataset_pipeline_shuffle_buffer_size)
dataset = dataset.prefetch(hp.dataset_pipeline_prefetch)
dataset = dataset.batch(hp.batch_size)

dataset_iterator = dataset.make_initializable_iterator()
(input_sequences_it,
 input_lengths_it,
 input_word_endings_it,
 target_sequences_it) = dataset_iterator.get_next()

# Placeholders
# ------------

input_sequences = tf.placeholder_with_default(input_sequences_it,
                                              shape = [None, hp.max_sequence_length],
                                              name = 'input_sequences')
input_lengths = tf.placeholder_with_default(input_lengths_it,
                                            shape = [None],
                                            name = 'input_lengths')
input_word_endings = tf.placeholder_with_default(input_word_endings_it,
                                                 shape = [None, hp.max_sequence_length],
                                                 name = 'input_word_endings')
target_sequences = tf.placeholder_with_default(target_sequences_it,
                                               shape = [None, hp.max_sequence_length],
                                               name = 'target_sequences')

# sequences of input positions (not a placeholder)
input_positions = tf.range(hp.max_sequence_length, dtype=tf.int32)
input_positions = tf.tile(input_positions, [tf.shape(input_sequences)[0]])
input_positions = tf.reshape(input_positions, 
                             (tf.shape(input_sequences)[0], hp.max_sequence_length), 
                             name = 'input_positions')

is_training = tf.placeholder(tf.bool)


# Embeddings
# ----------

input_sequence_embeddings = tf.get_variable('input_sequence_embeddings', 
                                            (hp.vocab_size, hp.embedding_size))
input_sequences_embedded = tf.nn.embedding_lookup(input_sequence_embeddings, 
                                                  input_sequences,
                                                  name = 'input_sequences_embedded')

input_position_embeddings = tf.get_variable('input_position_embeddings', 
                                            (hp.max_sequence_length, hp.embedding_size))
input_positions_embedded = tf.nn.embedding_lookup(input_position_embeddings, input_positions)

input_word_ending_embeddings = tf.get_variable('input_word_ending_embeddings',
                                               (2, hp.embedding_size))
input_word_endings_embedded = tf.nn.embedding_lookup(input_word_ending_embeddings, 
                                                     input_word_endings,
                                                     name = 'input_word_endings_embedded')

input_combined_embedded = tf.add_n([input_sequences_embedded, 
                                    input_positions_embedded, 
                                    input_word_endings_embedded])
input_combined_embedded = tf.layers.dropout(input_combined_embedded,
                                            rate = hp.dropout_rate,
                                            training = is_training)

# RNNs
# ----

dropout_keep_prob = tf.cond(is_training,
                            lambda: tf.constant(1.0),
                            lambda: tf.constant(1.0 - hp.dropout_rate))

fw_rnn_cell = tf.nn.rnn_cell.GRUCell(hp.rnn_hidden_state_size)
fw_rnn_cell = tf.nn.rnn_cell.DropoutWrapper(fw_rnn_cell,
                                            input_keep_prob = dropout_keep_prob,
                                            output_keep_prob = dropout_keep_prob)

bw_rnn_cell = tf.nn.rnn_cell.GRUCell(hp.rnn_hidden_state_size)
bw_rnn_cell = tf.nn.rnn_cell.DropoutWrapper(bw_rnn_cell,
                                            input_keep_prob = dropout_keep_prob,
                                            output_keep_prob = dropout_keep_prob)

rnn_outputs, _ = tf.nn.bidirectional_dynamic_rnn(fw_rnn_cell,
                                                 bw_rnn_cell,
                                                 input_combined_embedded,
                                                 sequence_length = input_lengths,
                                                 dtype = tf.float32)
rnn_outputs = tf.concat(rnn_outputs, 2)

# Feed-forward
# ------------

# TODO: more layers here?
feed_forward = tf.layers.dense(rnn_outputs,
                               hp.ff_hidden_state_size,
                               activation = tf.nn.relu,
                               name = 'feed_forward')
feed_forward = tf.layers.dropout(feed_forward,
                                 rate = hp.dropout_rate,
                                 training = is_training)


# Mask
# ----

sequence_mask = tf.sequence_mask(input_lengths,
                                 hp.max_sequence_length,
                                 dtype = tf.bool)


# Softmax
# -------

output_logits = tf.layers.dense(feed_forward, 2, name='softmax')

# TODO: is the softmax here really necessary?
output_sequences = tf.nn.softmax(output_logits)
output_sequences = tf.argmax(output_sequences, axis = -1)
output_sequences *= tf.cast(sequence_mask, tf.int64)

# Loss
# ----

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target_sequences,
                                                        logits = output_logits)
losses *= tf.cast(sequence_mask, tf.float32)

total_loss = tf.reduce_sum(losses)
total_input_length = tf.reduce_sum(input_lengths)
mean_loss  = total_loss / tf.cast(total_input_length, tf.float32)

true_positives = tf.reduce_sum(output_sequences * target_sequences)
false_positives = tf.reduce_sum(tf.maximum(output_sequences - target_sequences, 0))
false_negatives = tf.reduce_sum(tf.maximum(target_sequences - output_sequences, 0))

# Training
# --------

global_step = tf.Variable(0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train_op = optimizer.minimize(mean_loss, global_step=global_step)

# Summary
# -------

total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    print('parameters for "%s": %d' % (variable.name, variable_parameters))
    total_parameters += variable_parameters
print('total parameters: %d' % total_parameters)

parameters for "input_sequence_embeddings:0": 2560000
parameters for "input_position_embeddings:0": 10240
parameters for "input_word_ending_embeddings:0": 512
parameters for "bidirectional_rnn/fw/gru_cell/gates/kernel:0": 262144
parameters for "bidirectional_rnn/fw/gru_cell/gates/bias:0": 512
parameters for "bidirectional_rnn/fw/gru_cell/candidate/kernel:0": 131072
parameters for "bidirectional_rnn/fw/gru_cell/candidate/bias:0": 256
parameters for "bidirectional_rnn/bw/gru_cell/gates/kernel:0": 262144
parameters for "bidirectional_rnn/bw/gru_cell/gates/bias:0": 512
parameters for "bidirectional_rnn/bw/gru_cell/candidate/kernel:0": 131072
parameters for "bidirectional_rnn/bw/gru_cell/candidate/bias:0": 256
parameters for "feed_forward/kernel:0": 262144
parameters for "feed_forward/bias:0": 512
parameters for "softmax/kernel:0": 1024
parameters for "softmax/bias:0": 2
total parameters: 3622402


In [7]:
def evaluate_dataset(filename, header='results', train=False, show_progress=True):
    cum_loss = 0
    cum_input_length = 0
    
    cum_true_positives = 0
    cum_false_positives = 0
    cum_false_negatives = 0
    
    sess.run(dataset_iterator.initializer, feed_dict={
        dataset_filenames: [filename]
    })
    
    if show_progress:
        progress = tqdm()
        
    while True:
        try:
            (_,
             curr_loss, 
             curr_input_length, 
             curr_true_positives,
             curr_false_positives,
             curr_false_negatives) = sess.run((train_op if train else [],
                                               total_loss,
                                               total_input_length,
                                               true_positives,
                                               false_positives,
                                               false_negatives),
                                              feed_dict = { is_training: train })
        except tf.errors.OutOfRangeError:
            break

        if show_progress:
            progress.update(curr_input_length)

        cum_loss += curr_loss
        cum_input_length += curr_input_length
        cum_true_positives += curr_true_positives
        cum_false_positives += curr_false_positives
        cum_false_negatives += curr_false_negatives
    
    if show_progress:
        progress.close()

    precision = cum_true_positives / (cum_true_positives + cum_false_positives)
    recall = cum_true_positives / (cum_true_positives + cum_false_negatives)
    F1 = 2 * (precision * recall) / (precision + recall)
    
    print('%s: loss=%g, precision=%g, recall=%g, F1=%g' % (header,
                                                           cum_loss/cum_input_length, 
                                                           precision, 
                                                           recall, 
                                                           F1))

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    result = evaluate_dataset('../data/simplewiki/simplewiki-20171103.entity_recognition.train.tfrecords',
                              header='train %d' % epoch,
                              train=True,
                              show_progress=True)
    result = evaluate_dataset('../data/simplewiki/simplewiki-20171103.entity_recognition.dev.tfrecords',
                              header='dev %d' % epoch,
                              train=False,
                              show_progress=False)

23718333it [04:39, 84757.77it/s] 


train 0: loss=0.166985, precision=0.701448, recall=0.333386, F1=0.451963


0it [00:00, ?it/s]

dev 0: loss=0.163846, precision=0.777298, recall=0.300349, F1=0.433279


23718333it [04:22, 90373.30it/s] 


...

23718333it [04:16, 92610.74it/s] 


train 62: loss=0.107474, precision=0.810831, recall=0.634501, F1=0.71191


0it [00:00, ?it/s]

dev 62: loss=0.187137, precision=0.637214, recall=0.479547, F1=0.547251


12191419it [02:08, 94210.27it/s] 